In [1]:
# Imports
import os, sys, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
print('TensorFlow:', tf.__version__)
print('GPUs:', tf.config.list_physical_devices('GPU'))

TensorFlow: 2.20.0
GPUs: []


In [4]:
# Paths & Hyperparameters
BASE_PATH = '../data/brain_mri/'
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
VAL_PATH = os.path.join(BASE_PATH, 'val')
TEST_PATH = os.path.join(BASE_PATH, 'test')
for p in [TRAIN_PATH, VAL_PATH, TEST_PATH]:
    assert os.path.isdir(p), f'Missing directory: {p}'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_PHASE_1 = 15  # keep moderate for demo; adjust as needed
EPOCHS_PHASE_2 = 15
MODELS_DIR = '../models'
os.makedirs(MODELS_DIR, exist_ok=True)

In [5]:
# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True
)
validation_generator = val_test_datagen.flow_from_directory(
    VAL_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)
test_generator = val_test_datagen.flow_from_directory(
    TEST_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)
num_classes = train_generator.num_classes
class_indices = train_generator.class_indices
idx_to_class = {v: k for k, v in class_indices.items()}
print('Classes:', class_indices)

FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
# Class Weights
classes_unique = np.unique(train_generator.classes)
cw = compute_class_weight(class_weight='balanced', classes=classes_unique, y=train_generator.classes)
class_weights = {int(k): float(v) for k, v in zip(classes_unique, cw)}
print('Class Weights:', class_weights)

In [ ]:
# Build Model (Phase 1 - frozen backbone)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Phase 1 Training - Transfer Learning
Train only classification head until validation stabilizes or max epochs reached.

In [ ]:
# Callbacks Phase 1
best_phase1 = os.path.join(MODELS_DIR, 'brain_mri_densenet_phase1_best.h5')
ckpt_each_p1 = os.path.join(MODELS_DIR, 'brain_mri_phase1_epoch_{epoch:02d}_valacc_{val_accuracy:.4f}.h5')
callbacks_p1 = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint(best_phase1, monitor='val_accuracy', save_best_only=True, verbose=1),
    ModelCheckpoint(ckpt_each_p1, monitor='val_accuracy', save_best_only=False, verbose=0)
]
history_p1 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE_1,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=callbacks_p1,
    verbose=1
)

### Phase 2 Training - Fine-tuning
Unfreeze upper half of DenseNet-121 and continue training with lower learning rate.

In [ ]:
# Fine-tuning setup
base_model.trainable = True
fine_tune_at = len(base_model.layers) // 2
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
best_phase2 = os.path.join(MODELS_DIR, 'brain_mri_densenet_phase2_best.h5')
ckpt_each_p2 = os.path.join(MODELS_DIR, 'brain_mri_phase2_epoch_{epoch:02d}_valacc_{val_accuracy:.4f}.h5')
callbacks_p2 = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint(best_phase2, monitor='val_accuracy', save_best_only=True, verbose=1),
    ModelCheckpoint(ckpt_each_p2, monitor='val_accuracy', save_best_only=False, verbose=0)
]
history_p2 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE_1 + EPOCHS_PHASE_2,
    initial_epoch=len(history_p1.history['loss']),
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=callbacks_p2,
    verbose=1
)

### Evaluation on Test Set

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=0)
print(f'Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc*100:.2f}%')
probs = model.predict(test_generator, verbose=0)
y_true = test_generator.classes
y_pred = np.argmax(probs, axis=1)
cm = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=[idx_to_class[i] for i in range(num_classes)])
print(report)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[idx_to_class[i] for i in range(num_classes)], yticklabels=[idx_to_class[i] for i in range(num_classes)])
plt.title('Confusion Matrix - Brain MRI')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

### Training Curves

In [ ]:
acc = history_p1.history['accuracy'] + history_p2.history['accuracy']
val_acc = history_p1.history['val_accuracy'] + history_p2.history['val_accuracy']
loss = history_p1.history['loss'] + history_p2.history['loss']
val_loss = history_p1.history['val_loss'] + history_p2.history['val_loss']
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(acc, label='Train Acc')
plt.plot(val_acc, label='Val Acc')
plt.axvline(x=EPOCHS_PHASE_1-1, color='r', linestyle='--', label='Fine-tune start')
plt.legend(); plt.title('Accuracy'); plt.xlabel('Epoch')
plt.subplot(1,2,2)
plt.plot(loss, label='Train Loss')
plt.plot(val_loss, label='Val Loss')
plt.axvline(x=EPOCHS_PHASE_1-1, color='r', linestyle='--', label='Fine-tune start')
plt.legend(); plt.title('Loss'); plt.xlabel('Epoch')
plt.tight_layout(); plt.show()

### Grad-CAM Visualization

In [ ]:
# Simple Grad-CAM for a few test samples
def get_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
# find last conv layer
last_conv = None
for layer in model.layers[::-1]:
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv = layer.name; break
if last_conv is None:
N = min(3, len(test_generator.filenames))
for i in range(N):
    batch = test_generator[i]
print('Grad-CAM done.')

### Save Final Model

In [ ]:
final_model_h5 = os.path.join(MODELS_DIR, 'brain_mri_densenet_final.h5')
model.save(final_model_h5)
print('Saved:', final_model_h5)